# Neste notebook vamos definir as funçoes de filtragem e visualizaçao dos dados a partir das classes definidas na bilbioteca GeoPandas e Matplotlib

[Mapping and Plotting Tools](https://geopandas.org/docs/user_guide/mapping.html)

[Matplotlib](https://matplotlib.org/stable/tutorials/introductory/usage.html#sphx-glr-tutorials-introductory-usage-py)

- Com as funçoes definidas e testadas neste notebook, criaremos um arquivo .py no diretorio .soureces/ para armazena-las, possibilitando assim, que possamos chama-las com a funçao import.

In [1]:
import geopandas as gpd
import pandas as pd
#import fiona

In [2]:
import matplotlib.pyplot as plt

In [3]:
# Origem do banco de dados
gdb = '/home/ggrl/geodatabase/'

In [4]:
# Parametro para plot interativo
%matplotlib widget

## Importando um vetor do banco de dados criado no notebook passado.
- Escolha de camada e mapa
    - Caso nao tenha o mapa selecionado retorne lista de mapas disponiveis para aquela escala.
    - Caso nao escolha um mapa retorne todos os vetores daquela camada.

In [5]:
# IMPORTADOR DE LITOLOGIAS POR ESCALA --------------------------------------------------------------------------#
def importar_geometrias(camada, mapa=False):
    '''
    Recebe:
        camada      : camada vetorial a ser lida do geopackage.
        mapa        : nome do mapa presente na camada vetorial;

    Retorna:
        Objeto GeoDataFrame.
        
    Se houver seleçao de mapa retornara apenas as geometrias que possuem o nome escolhido na coluna ['MAPA']
    Se Retornar camada vazia recebera a lista das camadas veotoriais diposniveis
    Se mapa == False: retorna todos os objetos presente nesta camada vetorial
    '''
    lito =  gpd.read_file(gdb+'geodatabase.gpkg',
                        driver= 'GPKG',
                        layer= camada)
                        
    #print(f"# -- Lista de camadas vetoriais disponiveis: {list(fiona.listlayers(gdb+'geodatabase.gpkg'))}")  

    if mapa:
        folha = lito[lito.MAPA == 'Carta geológica da folha '+mapa]
        if folha.empty:
            print(f"O mapa escolhido nao est'a presente na coluna MAPA da camada veotiral.")
            print(f"# -- Lista de mapas: {list(lito.MAPA.unique())}")

        else:    
            return(folha)
    else:
        return(lito)

###  Utilizando as funçoes definidas acima para importar os dados

In [6]:
# Abrindo os vetores descritivos dos projetos aerogeofisicos
proj_geof = importar_geometrias('proj_aerogeof')

In [7]:
litologia_1kk = importar_geometrias('litologia_1kk')

In [8]:
litologia_250k = importar_geometrias('litologia_250k')

In [32]:
litologia_100k = importar_geometrias('litologia_100k')

In [33]:
litologia_50k = importar_geometrias('litologia_50k')

## Descrevendo as caracteristicas dos dados

In [9]:
#from shapely.ops import cascaded_union
#area_250k = gpd.GeoSeries(cascaded_union(litologia_250k['geometry']))

In [82]:
litologia_50k.head(5)

,ID_UNIDADE,SIGLA,HIERARQUIA,NOME,AMBIENTE_T,SUB_AMBIEN,SIGLA_PAI,NOME_PAI,LEGENDA,ESCALA,...,ERA_MIN,ERA_MAX,SISTEMA_MI,SISTEMA_MA,EPOCA_MIN,EPOCA_MAX,SIGLAS_HIS,SHAPE_AREA,SHAPE_LEN,geometry
0,5309,K_lambda_bam,Litofácies,"Litofácies Banhadão, flogopita melteigito",Magmatismo intraplaca,Intrusivas alcalinas,K_lambda_ba,K_lambda_ba - Suíte Alcalina Banhadão,Flogopita melteigito,1:50.000,...,Mesozóico,Mesozóico,Cretáceo,Cretáceo,Inferior,Inferior,None,0.000005,0.013937,"POLYGON ((-49.38349 -24.65607, -49.38344 -24.6..."
1,5324,MP1aca,Litofácies,"Litofácies Água Clara, anfibolito",Orógeno,Terreno de arco de ilhas,MP1ac,MP1ac - Formação Água Clara,Lentes de anfibolito cinza escuro-esverdeado e...,1:50.000,...,Mesoproterozóico,Mesoproterozóico,Calimiano,Calimiano,Médio,Médio,None,0.000038,0.036405,"POLYGON ((-49.22189 -24.72646, -49.22225 -24.7..."
2,5324,MP1aca,Litofácies,"Litofácies Água Clara, anfibolito",Orógeno,Terreno de arco de ilhas,MP1ac,MP1ac - Formação Água Clara,Lentes de anfibolito cinza escuro-esverdeado e...,1:50.000,...,Mesoproterozóico,Mesoproterozóico,Calimiano,Calimiano,Médio,Médio,None,0.000006,0.011015,"POLYGON ((-49.16267 -24.56268, -49.16273 -24.5..."
3,1696,MP1acx,Litofácies,"Litofácies Água Clara, xisto",None,None,MP1ac,MP1ac - Formação Água Clara,Predomínio de mica xistos e filitos grafitosos...,1:50.000,...,Mesoproterozóico,Mesoproterozóico,Calimiano,Calimiano,Médio,Médio,None,0.000156,0.102974,"POLYGON ((-49.16267 -24.56268, -49.16266 -24.5..."
4,1696,MP1acx,Litofácies,"Litofácies Água Clara, xisto",None,None,MP1ac,MP1ac - Formação Água Clara,Predomínio de mica xistos e filitos grafitosos...,1:50.000,...,Mesoproterozóico,Mesoproterozóico,Calimiano,Calimiano,Médio,Médio,None,0.000008,0.026624,"POLYGON ((-49.15930 -24.59079, -49.15945 -24.5..."


In [98]:
area_lito_1kk = litologia_1kk[:].area.sum()

/tmp/ipykernel_13466/4150503765.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  area_lito_1kk = litologia_1kk[:].area.sum()


In [103]:
area_geof_500m = proj_geof_500m[:].area.sum()
area_geof_1000m = proj_geof_1000m[:].area.sum()

/tmp/ipykernel_13466/3075464236.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  area_geof_500m = proj_geof_500m[:].area.sum()
/tmp/ipykernel_13466/3075464236.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  area_geof_1000m = proj_geof_1000m[:].area.sum()


In [105]:
(area_geof_500m/area_lito_1kk) * 100

46.597996653597654

In [104]:
(area_geof_1000m/area_lito_1kk) * 100

14.41795076030269

In [35]:
len(litologia_100k['MAPA'].unique())

311

In [72]:
meta_lito_100k = pd.DataFrame(litologia_100k.dtypes)
meta_lito_100k.rename(columns={0:"dtype"},inplace=True)

meta_lito_100k['Null'] = litologia_100k.isnull().sum()

meta_lito_100k['Unique Values'] = litologia_100k.nunique()

In [73]:
#meta_lito_1kk
meta_lito_100k

,dtype,Null,Unique Values
ID_UNIDADE,int64,0,2837
SIGLA,object,0,2837
HIERARQUIA,object,0,31
NOME,object,0,2829
AMBIENTE_T,object,47225,10
SUB_AMBIEN,object,53942,33
SIGLA_PAI,object,31377,599
NOME_PAI,object,31377,599
LEGENDA,object,378,2724
ESCALA,object,0,1


In [17]:
proj_geof_1000m = proj_geof[proj_geof['ESPACAMENT'] == '1000']
proj_geof_500m = proj_geof[proj_geof['ESPACAMENT'] == '500']

In [18]:
proj_geof_500m.columns

Index(['ID_PROJETO', 'TITULO', 'METODOS', 'ALTURA_VOO', 'AREA_LEVAN',
       'QUILOMETRA', 'DIRECAO_LV', 'DIRECAO_LC', 'ESPACAMENT', 'ESPACAME_1',
       'ID_SERIE', 'SERIE', 'DISPONIBIL', 'IMAGENS', 'SHAPE_AREA', 'SHAPE_LEN',
       'geometry'],
      dtype='object')

In [19]:
proj_geof_500m = proj_geof_500m.loc[:, ('ID_PROJETO', 'TITULO', 'METODOS', 'ALTURA_VOO', 'AREA_LEVAN','QUILOMETRA', 'DIRECAO_LV', 'DIRECAO_LC',
                                        'ESPACAMENT', 'ESPACAME_1','ID_SERIE', 'SERIE', 'DISPONIBIL', 'IMAGENS', 'SHAPE_AREA', 'SHAPE_LEN','geometry')]

In [20]:
print(proj_geof_1000m['SHAPE_AREA'].sum())
print(proj_geof_500m['SHAPE_AREA'].sum())

100.8120406558254
325.81878046663854


In [74]:
ax_1kk = litologia_1kk.plot('SIGLA')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [78]:
proj_geof_1000m.boundary.plot(ax=ax_1kk,color='blue')
proj_geof_500m.boundary.plot(ax=ax_1kk,color='red')

plt.title('Area de Cobertura dos Dados')

Text(0.5, 1.0, 'Area de Cobertura dos Dados')

In [75]:
ax_250k = litologia_250k.plot('SIGLA')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [76]:
ax_100k = litologia_100k.plot('SIGLA')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
proj_geof[proj_geof['ESPACAMENT'].astype(int)]

In [ ]:
proj_geof_500m['ESPACAMENT'] = proj_geof_500m['ESPACAMENT']

In [ ]:
proj_geof_500m.reset_index(drop=True,inplace=True)

In [ ]:
proj_geof_500m['ESPACAMENT']

In [ ]:
fig, axs = plt.subplots()
ax.plot(proj_geof['geometry'],label='proj')

## Calculando area de cobertura dos projetos aerogeofisicos

In [ ]:
a = litologia_100k.bounds

In [ ]:
a

In [ ]:
a['minx']

In [ ]:
a.plot()

In [ ]:
#b = a.unary_union

In [ ]:
proj_250m = proj_geof[proj_geof['ESPACAMENT'] == '250']

In [ ]:
print(proj_250m['geometry'][16])

In [ ]:
litologia_100k

In [ ]:
proj_geof_500m = proj_geof[proj_geof['ESPACAMENT'] == '500']

In [ ]:
print(len(proj_geof_500m))
proj_geof_500m['DISPONIBIL'].isnull().sum()

In [ ]:
proj_geof_500m.columns

In [ ]:
dic_meta_litologico = {'lista_colunas' : '',}
dic_meta_geofisico = {'lista_colunas' : '',}

In [ ]:
print(dic_meta_litologico.keys())

In [ ]:
x = list(proj_geof_500m.columns)
dic_meta_litologico.update({'lista_colunas':x})

In [ ]:
print(dic_meta_litologico['lista_colunas'])

In [ ]:
dic_meta_litologico['lista_colunas']

In [ ]:
#proj_geof_500m['SHAPE_AREA'].sum() / 

In [ ]:
proj_geof.columns

In [ ]:
#proj_geof.geometry.area.sum()

In [ ]:
print(type(proj_geof['ESPACAMENT'][0]))
print(list(proj_geof.ESPACAMENT.unique()))
print(list(proj_geof.ESPACAMENT.unique()))

In [ ]:
lista_espacament = list(proj_geof.ESPACAMENT.unique())

In [ ]:
print(f"Foram identificados {len(proj_geof[proj_geof['ESPACAMENT'] == '500'])} projetos aerogeofisicos")
print(f"Lista de Titulos: {proj_geof[proj_geof['ESPACAMENT'] == '500']['TITULO']}")

In [ ]:
proj_geof_500m = proj_geof[proj_geof['ESPACAMENT'] == '500']

In [ ]:
# Abrindo os vetores litológicos utilizando a função definida acima
litologia_100k = importar_geometrias('litologia_100k')
litologia_250k = importar_geometrias('litologia_250k')

In [ ]:
plot_brazil(proj_geof_500m.boundary)
plt.suptitle('500 metros')
plt.annotate('ID_PROJETO',)
plt.axis('scaled')

In [ ]:
#int(proj_geof['ESPACAMENT'])

In [ ]:
proj_geof_500m = proj_geof[proj_geof.ESPACAMENT == '500']

In [ ]:
#ax = litologia_100k.plot('SIGLA')
#proj_geof_500m.boundary.plot(ax=ax)

plot_base(proj_geof_500m.boundary,camada='litologia_100k')  # esta funçao e lenta pois importa toda a camada vetorial novamente e
                                                            # nao deve ser usada caso a litologia ja tenhasido definida

## Listando atributos geologicos das geometrias importadas.

In [ ]:
# Abrindo os vetores litológicos utilizando a função definida acima
litologia_100k = importar_geologico('litologia_100k')
litologia_250k = importar_geologico('litologia_250k')

In [ ]:
# Abrindo os vetores litológicos de um unico mapa utilizando a função definida acima
varginha_100k = importar_geologico('litologia_100k','Varginha - UFRJ')

In [ ]:
# atributos descritivos de cada uma das camadas litológicas (geometrias)
print("Lista de atributos descritivos classes litologicas: ")
print(list(varginha_100k.columns))

In [ ]:
# Plotando os vetores descritos pelos geólogos da cprm
varginha_100k.plot('SIGLA')

In [ ]:
print(varginha_100k.shape)
print(f"{varginha_100k.shape[0]} geometrias litologicas descritas em {varginha_100k.shape[1]} atributos")

In [ ]:
# Observando o formato da descrição da coluna litotipos
varginha_100k_litotipos = varginha_100k.LITOTIPOS.unique()
print(f"{len(list(varginha_100k_litotipos))} litotipos reconhecidos")
list(varginha_100k_litotipos)

In [ ]:
# com estas funçoes utilizadas asssimas, podemos definir uma funçao que descreve o nosso dado vetorial
def describe_geologico(gdf):
    lista_colunas = list(gdf.columns)
    lista_shape = gdf.shape
    lista_litotipos = list(gdf.LITOTIPOS.unique())
    lista_legenda = list(gdf.LEGENDA.unique())

    dic_litologico = {'lista_colunas': lista_colunas,
                      'lista_shape': lista_shape,
                      'lista_litotipos': lista_litotipos,
                      'lista_legenda': lista_legenda}
    return dic_litologico   

In [ ]:
dic_litoligico = describe_geologico(varginha_100k)

In [ ]:
print(f"Numero de geometrias descritas na carta geologica: {len(dic_litoligico['lista_litotipos'])}")
print("Lista de valores unicos da coluna Litotipo: ")
list(dic_litoligico['lista_litotipos'])

In [ ]:
print(f"Numero de geometrias descritas na carta geologica: {len(dic_litoligico['lista_legenda'])}")
print("Lista de valores unicos da coluna Legenda: ")
list(dic_litoligico['lista_legenda'])

## Reconhecendo as litologia mapeadas em suas escalas
- Faremos um filtro na coluna LEGENDA e LITOTIOPOS para selecionar as classes de interesse e, com estas geometrias, selecionaremos os mapas que as contém.
- Assim podemos identificar quais cartas geológicas e suas escalas podemos trabalhar.

In [ ]:
def filtro(gdf,mineral):
    filtrado = gdf[gdf['LITOTIPOS'].str.contains(mineral)]
    if filtrado.empty:
        print(f"{list(gdf['LITOTIPOS'].unique())}")
    else:
        return filtrado

In [ ]:
litologia_grafitosa_100k = filtro(litologia_100k,'graf')

In [ ]:
litotipos_vazio = litologia_100k[litologia_100k.LITOTIPOS.isna()]
litotipos_vazio.LITOTIPOS

In [ ]:
#Foi necessário inserir a str 'Vazio' em celas que não continham valores para que a função de filtro funcionasse corretamente na coluna LEGENDA.

#litologia_1kk.fillna('Vazio',inplace=True)
#litologia_250k.fillna('Vazio',inplace=True)
litologia_100k.fillna('Vazio',inplace=True)
#litologia_50k.fillna('Vazio',inplace=True)

In [ ]:
Grafita_l1kk = filtro(litologia_1kk,'Graf')
print(list(Grafita_l1kk['LITOTIPOS'].unique()))

In [ ]:
Grafita_l1kk.plot('SIGLA')

In [ ]:
base = brazil.boundary.plot(color='black',figsize=(12,12))

leg_grafitosa_l1kk.plot(ax=base,facecolor='grey')

lit_grafitoso_1kk.plot(ax=base,facecolor='black')

In [ ]:
list_gr_100k=mapas_lito_gr_100k
indice=0
for i in mapas_lito_gr_100k:
    print(i)
    list_gr_100k[indice] = l_100k[l_100k['MAPA'].str.contains("i")]
    indice+=1


In [ ]:
len(list_gr_100k)


### Desafio desnecessário para provar meus conhecimentos com for loops:

In [ ]:
lista_mapa = list(litologia_1kk.MAPA.unique())
print(len(lista_mapa))
lista_nome=[]

for mapa in lista_mapa:
    trash=0
    Mapa=[]
    for letra in mapa:
        if trash<25:
            trash+=1
        else:
            Mapa+=letra
    lista_nome += [''.join(Mapa)]

In [ ]:
lista_nome

In [ ]:
folha=lista_nome
indice=0
for i in lista_nome:
    print(i)
    folha[indice] = litologia_1kk[l_1kk['MAPA'].str.contains(i)].dissolve('MAPA')
    indice+=1

In [ ]:
folha[31].plot()

In [ ]:
lista_mapa = list(litolo_100k.MAPA.unique())
lista_mapa

In [ ]:
lista_nome=[]
for mapa in lista_mapa:
    trash=0
    Mapa=[]
    for letra in mapa:
        if trash<25:
            trash+=1
        else:
            Mapa+=letra
    lista_nome += [''.join(Mapa)]

In [ ]:
len(lista_nome)

In [ ]:
folha=lista_nome
indice=0
for i in lista_nome:
    print(i)
    folha[indice] = l_100k[l_100k['MAPA'].str.contains(i)].dissolve('MAPA')
    indice+=1

In [ ]:
len(folha)